In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv 
from sqlalchemy import create_engine
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sys 
sys.path.append('../maths/') 
from baseball_stats import BasicPitching

In [8]:
k = ['playerID', 'birthYear', 'birthMonth', 'birthDay']
import_people = pd.read_csv('../datafiles/People.csv', encoding='latin-1', usecols=k)
import_pitching = pd.read_csv('../datafiles/Pitching.csv', encoding='latin-1')
people = import_people.copy()

#fix fucky dates
people['birthYear'] = people['birthYear'].fillna(1875).astype(int)
people['birthMonth'] = people['birthMonth'].fillna(1).astype(int)
people['birthDay'] = people['birthDay'].fillna(1).astype(int)

#create a birthdate column so we can calulate an age at start of season
people['birthdate'] = people.apply(lambda x: f"""{x['birthYear']}-{x['birthMonth']}-{x['birthDay']}""", axis=1)
people['birthdate'] = pd.to_datetime(people['birthdate'], errors='coerce')

#merge the people and batting dataframes
pitching = people.copy().merge(import_pitching, on='playerID', how ='inner')
pitching['season_start'] = pitching.apply(lambda x: f"""{x['yearID']}-04-01""", axis=1)
pitching['season_start'] = pd.to_datetime(pitching['season_start'], errors='coerce')
pitching['age'] = pitching.apply(lambda x: relativedelta(x['season_start'], x['birthdate']).years, axis=1)

pitching = pitching.drop(columns = ['birthYear', 'birthMonth', 'birthdate', 'birthDay', 'stint', 'teamID', 'lgID', 'season_start'])

data_cols = pitching.columns.difference(['playerID', 'yearID', 'age'])

pitching[data_cols] = pitching[data_cols].fillna(0)

pitching

,playerID,yearID,W,L,G,GS,CG,SHO,SV,IPouts,...,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP,age
0,aardsda01,2004,1,0,11,0,0,0,0,32,...,0,2.0,0,61.0,5,8,0.0,1.0,1.0,22
1,aardsda01,2006,3,0,45,0,0,0,0,159,...,1,1.0,0,225.0,9,25,1.0,3.0,2.0,24
2,aardsda01,2007,2,1,25,0,0,0,0,97,...,2,1.0,0,151.0,7,24,2.0,1.0,1.0,25
3,aardsda01,2008,4,2,47,0,0,0,0,146,...,3,5.0,0,228.0,7,32,3.0,2.0,4.0,26
4,aardsda01,2009,3,6,73,0,0,0,38,214,...,2,0.0,0,296.0,53,23,2.0,1.0,2.0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51363,venezan01,2023,0,0,2,0,0,0,0,7,...,0,0.0,0,12.0,1,2,0.0,0.0,0.0,25
51364,bowlajo01,2023,0,1,2,1,0,0,0,9,...,0,0.0,0,14.0,1,1,0.0,0.0,0.0,26
51365,cacerke01,2023,0,0,2,0,0,0,0,4,...,1,0.0,0,8.0,2,1,1.0,0.0,0.0,23
51366,gomezyo01,2023,0,0,1,0,0,0,0,6,...,0,1.0,0,8.0,1,0,0.0,0.0,0.0,23
